## seq2seq Slack Bot

![](https://cdn-images-1.medium.com/max/2560/1*1I2tTjCkMHlQ-r73eRn4ZQ.png)

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
link = "https://drive.google.com/open?id=1jL45nejl8bwqoT2tPRMLB-g8p0PvRa3n"

In [0]:
fluff, id = link.split('=')
import pandas as pd
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('movie_lines.txt')

In [0]:
import numpy as np
import pandas as pd
import string
import pickle
import operator
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1. Import Data

In [0]:
# .txtから会話データを取得する
import codecs

with codecs.open("movie_lines.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines = f.read().split("\n")
    conversations = []
    for line in lines:
        data = line.split(" +++$+++ ")
        conversations.append(data)

In [7]:
conversations[:6]

[['L1045', 'u0', 'm0', 'BIANCA', 'They do not!'],
 ['L1044', 'u2', 'm0', 'CAMERON', 'They do to!'],
 ['L985', 'u0', 'm0', 'BIANCA', 'I hope so.'],
 ['L984', 'u2', 'm0', 'CAMERON', 'She okay?'],
 ['L925', 'u0', 'm0', 'BIANCA', "Let's go."],
 ['L924', 'u2', 'm0', 'CAMERON', 'Wow']]

In [0]:
# idと会話だけ取り出す
chats = {}
for tokens in conversations:
    if len(tokens) > 4:
        idx = tokens[0][1:]
        chat = tokens[4]
        chats[int(idx)] = chat

In [9]:
chats

{1045: 'They do not!',
 1044: 'They do to!',
 985: 'I hope so.',
 984: 'She okay?',
 925: "Let's go.",
 924: 'Wow',
 872: "Okay -- you're gonna need to learn how to lie.",
 871: 'No',
 870: 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 869: 'Like my fear of wearing pastels?',
 868: 'The "real you".',
 867: 'What good stuff?',
 866: "I figured you'd get to the good stuff eventually.",
 865: 'Thank God!  If I had to hear one more story about your coiffure...',
 864: "Me.  This endless ...blonde babble. I'm like, boring myself.",
 863: 'What crap?',
 862: 'do you listen to this crap?',
 861: 'No...',
 860: 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 699: 'You always been this selfish?',
 698: 'But',
 697: "Then that's all you had to say.",
 696: 'Well, no...',
 695: "You never wanted to go out with 'me, did you?",
 694: 'I was?',
 693: 'I looked for you back at the party, but you

In [10]:
# idと会話をセットにする
sorted_chats = sorted(chats.items(), key = lambda x: x[0])
sorted_chats

[(49, 'Did you change your hair?'),
 (50, 'No.'),
 (51, 'You might wanna think about it'),
 (59, 'I missed you.'),
 (60, 'It says here you exposed yourself to a group of freshmen girls.'),
 (61, 'It was a bratwurst.  I was eating lunch.'),
 (62, 'With the teeth of your zipper?'),
 (63, 'You the new guy?'),
 (64, 'So they tell me...'),
 (65, "C'mon.  I'm supposed to give you the tour."),
 (66, 'So -- which Dakota you from?'),
 (67, "North, actually.  How'd you   ?"),
 (68, 'I was kidding. People actually live there?'),
 (69, "Yeah.  A couple.  We're outnumbered by the cows, though."),
 (70, 'How many people were in your old school?'),
 (71, 'Thirty-two.'),
 (72, 'Get out!'),
 (73, 'How many people go here?'),
 (74, 'Couple thousand. Most of them evil'),
 (77, "That I'm used to."),
 (78,
  'Yeah, but these guys have never seen a horse.  They just jack off to Clint Eastwood.'),
 (87, 'That girl -- I --'),
 (88, 'You burn, you pine, you perish?'),
 (89, 'Who is she?'),
 (90, "Bianca Stratf

In [0]:
# 会話のペアごとに辞書を作る { 会話セットid: [会話リスト] }
conves_dict = {}
counter = 1
conves_ids = []
for i in range(1, len(sorted_chats)+1):
    if i < len(sorted_chats):
        if (sorted_chats[i][0] - sorted_chats[i-1][0]) == 1:
            # 1つ前の会話の頭の文字がないのを確認
            if sorted_chats[i-1][1] not in conves_ids:
                conves_ids.append(sorted_chats[i-1][1])
            conves_ids.append(sorted_chats[i][1])
        elif (sorted_chats[i][0] - sorted_chats[i-1][0]) > 1:            
            conves_dict[counter] = conves_ids
            conves_ids = []
        counter += 1
    else:
        pass

In [12]:
conves_dict

{3: ['Did you change your hair?', 'No.', 'You might wanna think about it'],
 19: ['I missed you.',
  'It says here you exposed yourself to a group of freshmen girls.',
  'It was a bratwurst.  I was eating lunch.',
  'With the teeth of your zipper?',
  'You the new guy?',
  'So they tell me...',
  "C'mon.  I'm supposed to give you the tour.",
  'So -- which Dakota you from?',
  "North, actually.  How'd you   ?",
  'I was kidding. People actually live there?',
  "Yeah.  A couple.  We're outnumbered by the cows, though.",
  'How many people were in your old school?',
  'Thirty-two.',
  'Get out!',
  'How many people go here?',
  'Couple thousand. Most of them evil'],
 21: ["That I'm used to.",
  'Yeah, but these guys have never seen a horse.  They just jack off to Clint Eastwood.'],
 27: ['That girl -- I --',
  'You burn, you pine, you perish?',
  'Who is she?',
  "Bianca Stratford.  Sophomore. Don't even think about it",
  'Why not?',
  "I could start with your haircut, but it doesn't ma

In [0]:
context_and_target = []
for conves in conves_dict.values():
    # ペアがない会話は捨てる
    if len(conves) % 2 != 0:
        conves = conves[:-1]
    for i in range(0, len(conves), 2):
        context_and_target.append((conves[i], conves[i+1]))

In [14]:
# ペア完成
context_and_target[:5]

[('Did you change your hair?', 'No.'),
 ('I missed you.',
  'It says here you exposed yourself to a group of freshmen girls.'),
 ('It was a bratwurst.  I was eating lunch.',
  'With the teeth of your zipper?'),
 ('You the new guy?', 'So they tell me...'),
 ("C'mon.  I'm supposed to give you the tour.",
  'So -- which Dakota you from?')]

In [0]:
context, target = zip(*context_and_target)

In [0]:
context = list(context)
target = list(target)

In [17]:
context[:5]

['Did you change your hair?',
 'I missed you.',
 'It was a bratwurst.  I was eating lunch.',
 'You the new guy?',
 "C'mon.  I'm supposed to give you the tour."]

In [18]:
target[:20]

['No.',
 'It says here you exposed yourself to a group of freshmen girls.',
 'With the teeth of your zipper?',
 'So they tell me...',
 'So -- which Dakota you from?',
 'I was kidding. People actually live there?',
 'How many people were in your old school?',
 'Get out!',
 'Couple thousand. Most of them evil',
 'Yeah, but these guys have never seen a horse.  They just jack off to Clint Eastwood.',
 'You burn, you pine, you perish?',
 "Bianca Stratford.  Sophomore. Don't even think about it",
 "I could start with your haircut, but it doesn't matter.  She's not allowed to date until her older sister does.  And that's an impossibility.",
 'Expressing my opinion is not a terrorist action.',
 'I still maintain that he kicked himself in the balls.  I was merely a spectator.',
 'Tempestuous?',
 'Patrick Verona   Random skid.',
 "I'm sure he's completely incapable of doing anything that interesting.",
 'Block E?',
 'Just a little.']

## Step 2. Preprocessing for text data

In [0]:
# from my_seq2seq_text_cleanear import text_modifier, nonalpha_remover
import re
MAX_LEN = 12

In [0]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    
    return text

### 2-1. Clean Text

In [0]:
tidy_target = []
for conve in target:
    text = clean_text(conve)
    tidy_target.append(text)

In [22]:
tidy_target[:20]

['no',
 'it says here you exposed yourself to a group of freshmen girls',
 'with the teeth of your zipper',
 'so they tell me',
 'so  which dakota you from',
 'i was kidding people actually live there',
 'how many people were in your old school',
 'get out',
 'couple thousand most of them evil',
 'yeah but these guys have never seen a horse  they just jack off to clint eastwood',
 'you burn you pine you perish',
 'bianca stratford  sophomore do not even think about it',
 'i could start with your haircut but it does not matter  she is not allowed to date until her older sister does  and that is an impossibility',
 'expressing my opinion is not a terrorist action',
 'i still maintain that he kicked himself in the balls  i was merely a spectator',
 'tempestuous',
 'patrick verona   random skid',
 'i am sure he is completely incapable of doing anything that interesting',
 'block e',
 'just a little']

In [0]:
tidy_context = []
for conve in context:
    text = clean_text(conve)
    tidy_context.append(text)

In [24]:
tidy_context[:20]

['did you change your hair',
 'i missed you',
 'it was a bratwurst  i was eating lunch',
 'you the new guy',
 "c'mon  i am supposed to give you the tour",
 'north actually  how would you   ',
 'yeah  a couple  we are outnumbered by the cows though',
 'thirtytwo',
 'how many people go here',
 'that i am used to',
 'that girl  i ',
 'who is she',
 'why not',
 'katarina stratford  my my  you have been terrorizing ms blaise again',
 "well yes compared to your other choices of expression this year today's events are quite mild  by the way bobby rictor's gonad retrieval operation went quite well in case you are interested",
 'the point is kat  people perceive you as somewhat ',
 "who's that",
 'that is pat verona the one who was gone for a year i heard he was doing porn movies',
 'he always look so',
 'mandella eat  starving yourself is a very slow way to die']

In [0]:
with open('tidy_target.txt', 'w') as f:
    for item in tidy_target:
        f.write("%s\n" % item)

In [0]:
with open('tidy_context.txt', 'w') as f:
    for item in tidy_context:
        f.write("%s\n" % item)

In [0]:
# decoderのinputには<BOS>と<EOS>タグ
bos = "<BOS> "
eos = " <EOS>"
final_target = [bos + conve + eos for conve in tidy_target] 
encoder_inputs = tidy_context
decoder_inputs = final_target

### 2-2. MAKE VOCABRALY

In [0]:
# 一旦もともと辞書サイズを調べる
dictionary = []
full_text = encoder_inputs + decoder_inputs
for text in full_text:
    words = text.split()
    for i in range(0, len(words)):
        if words[i] not in dictionary:
            dictionary.append(words[i])

In [0]:
from keras.preprocessing.text import Tokenizer
VOCAB_SIZE = 14999
tokenizer = Tokenizer(num_words=VOCAB_SIZE)

In [30]:
# 辞書を作る
tokenizer.fit_on_texts(full_text)
word_index = tokenizer.word_index
len(word_index)

65283

In [0]:
# リバースした辞書を用意
index_word = {}
for k, v in word_index.items():
    index_word[v] = k

In [0]:
with open('tidy_context.txt', 'w') as f:
    for item in tidy_context:
        f.write("%s\n" % item)
from google.colab import files
files.download('RecursiveModelA.png')

In [0]:
encoder_inputs = tidy_context
decoder_inputs = final_target

### 2-3. ONE-HOT VECTORIZER

In [0]:
# 単語のシーケンスを作る np.arrayにする
encoder_sequences = tokenizer.texts_to_sequences(encoder_inputs)
# encider_sequences = np.array(encider_sequences)

In [0]:
# デコーダーデータ
decoder_sequences = tokenizer.texts_to_sequences(decoder_inputs)
# decoder_sequences = np.array(decoder_sequences)

In [0]:
with open('encoder_inputs.txt', 'w') as f:
    for item in encoder_inputs:
        f.write("%s\n" % item)
from google.colab import files
files.download('encoder_inputs.txt')

In [0]:
with open('decoder_inputs.txt', 'w') as f:
    for item in decoder_inputs:
        f.write("%s\n" % item)
from google.colab import files
files.download('decoder_inputs.txt')

In [0]:
with open('padded_encoder_sequences.txt', 'w') as f:
    for item in encoder_sequences:
        f.write("%s\n" % item)
from google.colab import files
files.download('padded_encoder_sequences.txt')

In [0]:
with open('padded_decoder_sequences.txt', 'w') as f:
    for item in decoder_sequences:
        f.write("%s\n" % item)
from google.colab import files
files.download('padded_decoder_sequences.txt')

In [0]:
num_samples = len(encoder_sequences)
decoder_output_data = np.zeros((num_samples, MAX_LEN, VOCAB_SIZE), dtype="float32")

In [0]:
# outputの３Dテンソル
for i, seqs in enumerate(decoder_inputs):
    for j, seq in enumerate(seqs):
        if j > 0:
            # j-1?
            decoder_output_data[i, j, seq] = 1.

In [1]:
ls

1_0306_chatobot3.ipynb        movie_lines.txt
decoder_inputs.txt            padded_decoder_sequences.txt
encoder_inputs.txt            padded_encoder_sequences.txt
glove.6B.50d.txt


In [11]:
import codecs
with codecs.open("padded_encoder_sequences.txt", "rb", encoding="utf-8", errors="ignore") as f:
    lines2 = f.read().split("\n")

In [12]:
lines2

['[40, 3, 436, 28, 621]',
 '[4, 950, 3]',
 '[10, 27, 8, 4, 27, 1107, 802]',
 '[3, 5, 186, 168]',
 '[662, 4, 22, 346, 6, 130, 3, 5, 2407]',
 '[1211, 388, 50, 36, 3]',
 '[78, 8, 439, 20, 14, 13613, 115, 5, 5355, 477]',
 '[7554]',
 '[50, 274, 116, 59, 46]',
 '[11, 4, 22, 265, 6]',
 '[11, 206, 4]',
 '[71, 7, 47]',
 '[64, 9]',
 '[11389, 29, 29, 3, 17, 82, 1752, 174]',
 '[61, 70, 3982, 6, 28, 154, 3146, 15, 3809, 23, 386, 3672, 2758, 14, 429, 9852, 115, 5, 99, 1202, 6007, 1587, 248, 429, 61, 16, 327, 3, 14, 668]',
 '[5, 333, 7, 4340, 116, 3, 66, 5490]',
 '[306, 11]',
 '[11, 7, 3248, 7894, 5, 57, 71, 27, 320, 24, 8, 386, 4, 254, 21, 27, 128, 5814, 979]',
 '[21, 163, 92, 43]',
 '[421, 3201, 226, 7, 8, 123, 947, 99, 6, 341]',
 '[11, 7, 23]',
 '[4, 842, 11, 5, 288, 15, 23, 223, 3606, 14, 11390, 7226, 34, 744, 226, 43, 3, 52, 31, 33, 1759, 3896, 7, 1234, 5, 6664, 15, 1064, 6960, 3, 14, 387, 666, 7895, 134, 260, 13, 3607, 5, 244, 15, 162, 71, 118, 123, 2223, 2464]',
 '[64, 12, 570, 39, 11, 163, 39

In [8]:
lines1

['did you change your hair',
 'i missed you',
 'it was a bratwurst  i was eating lunch',
 'you the new guy',
 "c'mon  i am supposed to give you the tour",
 'north actually  how would you   ',
 'yeah  a couple  we are outnumbered by the cows though',
 'thirtytwo',
 'how many people go here',
 'that i am used to',
 'that girl  i ',
 'who is she',
 'why not',
 'katarina stratford  my my  you have been terrorizing ms blaise again',
 "well yes compared to your other choices of expression this year today's events are quite mild  by the way bobby rictor's gonad retrieval operation went quite well in case you are interested",
 'the point is kat  people perceive you as somewhat ',
 "who's that",
 'that is pat verona the one who was gone for a year i heard he was doing porn movies',
 'he always look so',
 'mandella eat  starving yourself is a very slow way to die',
 'that is this',
 'i realize that the men of this fine institution are severely lacking but killing yourself so you can be with will

In [9]:
text = "cost  stuff like chat  i have never in my life had"
words = text.split()

In [10]:
words

['cost',
 'stuff',
 'like',
 'chat',
 'i',
 'have',
 'never',
 'in',
 'my',
 'life',
 'had']

In [ ]:
padded_decoder_sequences = []
    for line in lines:
        data = line.split()
        conversations.append(data)

In [5]:
len(lines)

143865

In [19]:
lines[0]

'[1, 32, 2]'

In [20]:
lines[0])

ValueError: invalid literal for int() with base 10: '[1, 32, 2]'

### 2-4. PADDING

In [1]:
from keras.preprocessing.sequence import pad_sequences
encoder_input_data = pad_sequences(encoder_sequences, maxlen=MAX_LEN, dtype='float32', padding='post', truncating='post')
decoder_input_data = pad_sequences(decoder_sequences, maxlen=MAX_LEN, dtype='float32', padding='post', truncating='post')

Using TensorFlow backend.


NameError: ignored

### 2-5. Word2Vec: pretrained glove vector

In [0]:
embeddings_index = {}
with open('glove.6B.50d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

print("Glove Loded!")

In [0]:
VOCAB_SIZE = len(word_index) + 1

In [0]:
embedding_dimention = 50
def embedding_matrix_creater(embedding_dimention, word_index):
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dimention))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

## Step 3. Build Seq2Seq Model

In [0]:
from keras.layers import Embedding
from keras.layers import Input, Dense, LSTM, TimeDistributed
from keras.models import Model

In [0]:
# load the pre-trained GloVe vectors into the embedding layer
embed_layer = Embedding(input_dim = vocab_size, output_dim = 50, trainable = True, )
embed_layer.build((None,))
embed_layer.set_weights([embedding_matrix])

# encoder and decoder gloabal LSTM variables with 300 units
LSTM_cell = LSTM(300, return_state = True)
LSTM_decoder = LSTM(300, return_state = True, return_sequences = True)
# final dense layer that uses TimeDistributed wrapper to generate 'vocab_size' softmax outputs for each time step in the decoder lstm
dense = TimeDistributed(Dense(vocab_size, activation = 'softmax'))

input_context = Input(shape = (maxLen, ), dtype = 'int32', name = 'input_context')
input_target = Input(shape = (maxLen, ), dtype = 'int32', name = 'input_target')

# pass the inputs into the embedding layer
input_ctx_embed = embed_layer(input_context)
input_tar_embed = embed_layer(input_target)

# pass the embeddings into the corresponding LSTM layers
encoder_lstm, context_h, context_c = LSTM_cell(input_ctx_embed)
# the decoder lstm uses the final states from the encoder lstm as the initial state
decoder_lstm, _, _ = LSTM_decoder(input_tar_embed, initial_state = [context_h, context_c],)

output = dense(vocab_size, activation = 'softmax')(decoder_lstm)

model = Model([input_context, input_target], output)

In [0]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

## Step 4. Training Model

In [0]:
model.fit([context_1, final_target_1], outs, epochs = 1000, batch_size = 12

## Step 5. Inference 

In [0]:
# the question asked to the chatbot
question = 'Hey! How are you doing?'

# preprocessing to make the data into the format required by the model, same as during training
a = question.split()
for pos,i in enumerate(a):
    a[pos] = re.sub('[^a-zA-Z0-9 .,?!]', '', i)
    a[pos]= re.sub(' +', ' ', i)
    a[pos] = re.sub('([\w]+)([,;.?!#&\'\"-]+)([\w]+)?', r'\1 \2 \3', i)
    if len(i.split()) > maxlen:
            a[pos] = (' ').join(a[pos].split()[:maxlen])
            if '.' in a[pos]:
                ind = a[pos].index('.')
                a[pos] = a[pos][:ind+1]
            if '?' in a[pos]:
                ind = a[pos].index('?')
                a[pos] = a[pos][:ind+1]
            if '!' in a[pos]:
                ind = a[pos].index('!')
                a[pos] = a[pos][:ind+1]

In [0]:
question = ' '.join(a).split()

# make the question into an array of the corresponding indexes
question = np.array([word_to_index[w] for w in question])

# pad sequences
question = sequence.pad_sequences([question], maxlen = 20)

# Keras model used to train, so that we define the variables (tensors) that ultimately go into the infernce model
input_context = Input(shape = (maxLen, ), dtype = 'int32', name = 'input_context')
input_target = Input(shape = (maxLen, ), dtype = 'int32', name = 'output_context')

input_ctx_embed = embed_layer(input_context)
input_tar_embed = embed_layer(input_target)

encoder_lstm, context_h, context_c = LSTM_encoder(input_ctx_embed)
decoder_lstm, h, _ = LSTM_decoder(input_tar_embed, initial_state = [context_h, context_c],)

output = dense(decoder_lstm)

# Define the model for the input (question). Returns the final state vectors of the encoder LSTM
context_model = Model(input_context, [context_h, context_c])

# define the inputs for the decoder LSTM
target_h = Input(shape = (300, ))
target_c = Input(shape = (300, ))

# the decoder LSTM. Takes in the embedding of the initial word passed as input into the decoder model (the 'BOS' tag), 
# along with the final states of the encoder model, to output the corresponding sequences for 'BOS', and the new LSTM states.  
target, h, c = LSTM_decoder(input_tar_embed, initial_state = [target_h, target_c])
output = dense(target)
target_model = Model([input_target, target_h, target_c], [output, h, c])

# pass in the question to the encoder LSTM, to get the final encoder states of the encoder LSTM
question_h, question_c = context_model.predict(question)

# initialize the answer that will be generated for the 'BOS' input. Since we have used pre-padding for padding sequences,
# the last token in the 'answer' variable is initialised with the index for 'BOS'.
answer = np.zeros((1, maxLen))
answer[0, -1] = word_to_index['BOS']

In [0]:
# i keeps track of the length of the generated answer. This won't allow the model to genrate sequences with more than 20 words.
i = 1

# make a new list to store the words generated at each time step
answer_1 = []

# flag to stop the model when 'EOS' tag is generated or when 20 time steps have passed.
flag = 0

# run the inference model
while flag != 1:
    # make predictions for the given input token and encoder states
    prediction, prediction_h, prediction_c = target_model.predict([answer, question_h, question_c])
    
    # from the generated predictions of shape (num_examples, maxLen, vocab_size), find the token with max probability
    token_arg = np.argmax(prediction[0, -1, :])
    
    # append the corresponding word of the index to the answer_1 list
    answer_1.append(index_to_word[token_arg])
    
    # set flag to 1 if 'EOS' token is generated or 20 time steps have passed
    if token_arg == word_to_index['EOS'] or i > 20:
        flag = 1
    # re-initialise the answer variable, and set the last token to the output of the current time step. This is then passed
    # as input to the next time step, along with the LSTM states of the current time step
    answer = np.zeros((1,maxLen))
    answer[0, -1] = token_arg
    question_h = prediction_h
    question_c = prediction_c
    
    # increment the count of the loop
    i+=1
    
 # print the answer generated for the given question
print (' '.join(answer_1))